# Generating empirical isochrones

## Aims

This iPython notebook generates polynomial fits to several well-studied nearby open clusters whose ages are well agreed upon. For the purpose of the 4MOST survey we aim to use polynomial fits for NGC2547 (38-41 Myr, Jeffries et al. 2005, MNRAS, 358, 13J -- herein Jeffries+05) and the Pleiades (125+/-20 Myr, Stauffer et al. 1998, ApJ, 499L, 199S) where the adopted ages are from the (almost totally) model-independent Lithium Depletion Boundary method. The fits represent empirical isochrones in the de-reddened, extinction-corrected colour-magnitude diagrams. The present colour-magnitude diagrams we have are:

(1) absolute G versus G-Ks

(2) absolute G versus Bp-Rp.

The code is structured such that other photometric filters can be easily included.

### To this aim, the goals of this notebook are to:
1. Collect homogeneous data for several clusters of known age.
2. Apply corrections for reddening, extinction and some minor calibrations to Gaia EDR3 data.
3. In each cluster, identify a set of presumably single stars for which a polynomial fit will be made in CMD space.

## Selecting clusters and cluster members.

The majority of the selected clusters (NGC 6530, $\gamma$ Vel, NGC2547 and NGC2516) were sourced from the 6th and final data relase of [the Gaia ESO Survey](https://www.gaia-eso.eu/), and selected targets have membership probabilities >95 % based on 3D kinematics using GES RVs and Gaia EDR3 astrometry -- see Table 3 in [Jackson et al. (2022, MNRAS, 509, 1664J)](https://academic.oup.com/mnras/article/509/2/1664/6414548). Pleiades targets are sourced from Olivares et al. 2018 (A&A 617, A15; Vizier: [J/A+A/617/A15/table1](https://vizier.u-strasbg.fr/viz-bin/VizieR?-source=J/A+A/617/A15&-to=3)) with a membership probability threshold >99%. An additional sample of 47 RV-confirmed, low-mass NGC2547 members are provided from Table 1 of Jeffries+05: Vizier: [J/MNRAS/358/13/table1](https://vizier.u-strasbg.fr/viz-bin/VizieR?-source=J/MNRAS/358/13&-to=3)) that were not present in the (unfiltered) GES table. Hyades targets are extracted from Oh & Evans [(2020, MNRAS, 498, 1920O)](https://academic.oup.com/mnras/article/498/2/1920/5899755).

Gaia EDR3 are collected using a simple TAP+ query with the Starlink/TOPCAT software with a search radii = 2.0". The vast majority (>99%) of targets lie within 0.2".

| Cluster | Age (Myr) | Filters | #sources | dmod | E(B-V) |
| --- | --- | --- | --- | --- | --- |
| NGC6530 | 1-2 | Pmem>95% | 303 (261, 300) | 10.60 ± 0.02 ± 0.09 | 0.44 ± 0.10 |
| gamVel | 15-20 | Pmem>95% | 189 (184, 186) | 7.73 ± 0.01 ± 0.02 | 0.04 ± 0.03 |
| NGC2547 | 38-41 | Pmem>95% or MemFlag={1,2} | 892 (768, 398) | 7.93 ± 0.01 ± 0.03 | 0.06 ± 0.03 |
| Pleiades | 125 ± 20 | P>99% | 193 (160, 162) | 5.65 ± 0.03 | 0.03 ± 0.01 |
| NGC2516 | 150 ± 30 | Pmem>95% | 479 (377, 471) | 8.07 ± 0.01 ± 0.03 | 0.11 ± 0.03 |
| Hyades | 625 ± 50 | Oh+20 | 1041 (867, 568) | Parallaxes | 0.01 ± 0.01 |

The table above shows the input parameters for the extinction and reddening corrections. Column 4 shows the number of sources that satisfy membership criteria, then the numbers in parethesis are those that are within the selected colour/magnitude range for CMDs (1) and (2) after applying the extinction/reddening corrections.

In both CMDs, targets have been filtered to remove source with large Bp-Rp photometric excesses as described by equation 18 in Riello et al. (2021, A&A 649, A3 -- Riello+21) and (where necessary) small Bp-Rp corrections (on the order of <0.02 mag) are applied using the coefficients supplied in table 2:

With the exception of the Hyades cluster (where individual parallaxes are used), cluster members are assumed to have a fixed cluster-value for their distance modulii and E(B-V).


In [5]:
import astropy.units as u
from astropy.coordinates import SkyCoord

from astropy.io import ascii
from astropy.table import Table
import numpy as np
from time import sleep
import matplotlib.pyplot as plt

## Correcting for the Bp-Rp flux excess
The following function, taken [directly from the Gaia EDR3 webpages](https://github.com/agabrown/gaiaedr3-flux-excess-correction/blob/main/FluxExcessFactorCorrectionCode.ipynb), applies the small correction due to flux excess in the Bp-Rp bands.

In [6]:
def correct_flux_excess_factor(bp_rp, phot_bp_rp_excess_factor):
    """
    Calculate the corrected flux excess factor for the input Gaia EDR3 data.
    
    Parameters
    ----------
    
    bp_rp: float, numpy.ndarray
        The (BP-RP) colour listed in the Gaia EDR3 archive.
    phot_bp_rp_excess_factor: float, numpy.ndarray
        The flux excess factor listed in the Gaia EDR3 archive.
        
    Returns
    -------
    
    The corrected value for the flux excess factor, which is zero for "normal" stars.
    
    Example
    -------
    
    phot_bp_rp_excess_factor_corr = correct_flux_excess_factor(bp_rp, phot_bp_rp_flux_excess_factor)
    """
    if np.isscalar(bp_rp) or np.isscalar(phot_bp_rp_excess_factor):
        bp_rp = np.float64(bp_rp)
        phot_bp_rp_excess_factor = np.float64(phot_bp_rp_excess_factor)
    
    if bp_rp.shape != phot_bp_rp_excess_factor.shape:
        raise ValueError('Function parameters must be of the same shape!')
        
    do_not_correct = np.isnan(bp_rp)
    bluerange = np.logical_not(do_not_correct) & (bp_rp < 0.5)
    greenrange = np.logical_not(do_not_correct) & (bp_rp >= 0.5) & (bp_rp < 4.0)
    redrange = np.logical_not(do_not_correct) & (bp_rp > 4.0)
    
    correction = np.zeros_like(bp_rp)
    correction[bluerange] = 1.154360 + 0.033772*bp_rp[bluerange] + 0.032277*np.power(bp_rp[bluerange], 2)
    correction[greenrange] = 1.162004 + 0.011464*bp_rp[greenrange] + 0.049255*np.power(bp_rp[greenrange], 2) \
        - 0.005879*np.power(bp_rp[greenrange], 3)
    correction[redrange] = 1.057572 + 0.140537*bp_rp[redrange]
    
    return phot_bp_rp_excess_factor - correction


## Removing sources with extreme Bp-Rp flux excess values
A simple function to remove sources whose flux excess lie outside of an n-sigma range, following section 9.4 in Riello+21.

In [7]:
def photclean(data,nsig):
    phot_excess = correct_flux_excess_factor(data['BPRP'],data['BPRP_excess'])
    condphot = np.abs(phot_excess) < nsig*(0.0059898+8.817481e-12*data['Gmag']**(7.618399))
    data["BPRPcorr"] = phot_excess
    dataclean = data[condphot]                                             
    return dataclean

## Quadrature sum of orthogonal error components
Some E(B-V) and distance modulus errors are broken down into statistical and systematic components. This function simply adds them in quadrature in case we need to use them at a later date.

In [8]:
def quad_sum(*args):
    return np.sqrt(np.sum([i**2 for i in args]))

## Converting E(B-V) to extinction/reddening corrections in various CMDs
This function uses the polynomial coefficients from the [table provided in the Gaia EDR3 documentation](https://www.cosmos.esa.int/web/gaia/edr3-extinction-law), reads in the required colour and magnitude needed and returns the corrections needed to apply the approriate reddening. 

In [9]:
def get_ext(A, c, m, p):
    p = np.array(list(p[0]))[:10].astype(float)
    return (p[0] + p[1]*c   + p[2]*c**2   + p[3]*c**3 +
                   p[4]*A   + p[5]*A**2   + p[6]*A**3 +
                   p[7]*A*c + p[8]*A*c**2 + p[9]*A**2*c)

## Return the polynomial fit
Simple function return the polynomial coefficients calculated for the selected single empirical sequence. 

In [10]:
def make_fit(n_order, poly, data):
    fit = 0
    for i in range(n_order+1):
        print(i, n_order-i)
        fit = fit + data**(n_order-i)*poly[i]
    return fit

## Setting up the colour-magnitude diagram
There are several processes involved in this function:
1. Calculate three separate extinction corrections: one for the magnitude, and the two colour components.
2. Declare a colour/magnitude range that will be used to parameterize the space over which the polynomial fit is made. The colour range is selected to 0.3 mag wider (at each end) than the {TS1, TS2} range set by Germano in the Target_Selection_Main_V1.03 notebook.
3. For each cluster, select stars that lie in this colour/magnitude space and apply a 2nd-order polynomial fit. The idea here is to discard the "bright" equal-mass unresolved multiple stars and select a sample of likely single objects to define the final polynomial fit. In some cases, like the Pleiades, this fit cuts seems to cut off the single sequence around the mid-K-type stars, so the curve is shifted brighter in absolution magnitude by a small amount (~0.35 mag).
4. For the stars that lie fainter than the 2nd order polynomial, apply a fourth order polynomial and return the coefficients. Just on visual assessment, a 4th order seems to work best because there seem to be about 4 deflections in the shape of the cluster sequences.
5. Pass the corrected colours and magnitudes and the polynomial coefficients to a plotting routine. 

In [11]:
def prepare_cmd(CMDtype, d_mod, ed_mod, EBV, eEBV, absM, col, plotcol1, plotcol2, magshift, clus_name, makeplot):
# Make reddening/extinction corrections using table provided in Gaia EDR3
    col_buff = 0.3
    if CMDtype == 'GKs':
        p1 = t_ext[(t_ext['Xname'] == 'GK') & (t_ext['Kname'] == 'kG')]
        p2 = t_ext[(t_ext['Xname'] == 'GK') & (t_ext['Kname'] == 'kG')]
        p3 = t_ext[(t_ext['Xname'] == 'GK') & (t_ext['Kname'] == 'kK')]
        limcol = [1.5-col_buff, 4.2+col_buff]
        limmag = [2.0,99]
        plt.xlabel(r"$(G-K_{\rm s})_{0}$")

    if CMDtype == 'GBPRP':
        p1 = t_ext[(t_ext['Xname'] == 'BPRP') & (t_ext['Kname'] == 'kG')]
        p2 = t_ext[(t_ext['Xname'] == 'BPRP') & (t_ext['Kname'] == 'kBP')]
        p3 = t_ext[(t_ext['Xname'] == 'BPRP') & (t_ext['Kname'] == 'kRP')]
        limcol = [0.9-col_buff,3.4+col_buff]
        limmag = [0.0,10.2]
        plt.xlabel(r"BP-RP")
    plt.ylabel(r"$G_{0}$")
    plt.xlim(limcol)

    A1 = np.array([get_ext(3.09*EBV, col[i], absM[i], p1) for i in range(len(absM))])
    A2 = np.array([get_ext(3.09*EBV, col[i], absM[i], p2) for i in range(len(absM))])
    A3 = np.array([get_ext(3.09*EBV, col[i], absM[i], p3) for i in range(len(absM))])
    mag0 = np.array(absM) - d_mod - A1
    col0 = np.array(col)-(A2-A3)

# Only want G6V -- M7V range for the CMD fit. This easily covers the
# colour range for the 4MOST selection, but not too large that we have
# to worry about MSTO or extremely (intrinsically) faint stars/BDs.
    colR = col0[(col0 > limcol[0]) & (col0 < limcol[1]) & (mag0 > limmag[0]) & (mag0 < limmag[1])]
    magR = mag0[(col0 > limcol[0]) & (col0 < limcol[1]) & (mag0 > limmag[0]) & (mag0 < limmag[1])]
# Sort colour/magnitude points in "increasing" colour ("decreasing" Teff).
    ind = np.argsort(colR)
    colR, magR = colR[ind], magR[ind]
    p = np.polyfit(colR, magR-0.35, 2)
    g = [magR >= np.polyval(p, colR)-magshift]
    p2= np.polyfit(colR[tuple(g)], magR[tuple(g)], 4)
    chi_squared = np.sum((np.polyval(p2, colR[g]) - magR[g]) ** 2)/(len(magR[g])-1)
    if makeplot == 1:
# Plot data-points
        ax.scatter(colR, magR, s=10, color=plotcol1, label=clus_name)

# second-order poly (and plot this)
        ax.plot(colR,np.polyval(p, colR)-magshift,color=plotcol2)

# sometimes it's good to shift this up and down by a few tenths of a
# magnitude to check we're getting enough stars in the "single"
# population and not having gaps in the calibrator points at any
# given colour (this is just done by eye).
        ax.plot(colR[g],np.polyval(p2, colR[g]), '--', color=plotcol2)
        return p2, colR, magR, chi_squared
    else:
        return p2, colR, magR, chi_squared

## Apply the Bp-Rp corrections to the cluster data (and remove any outliers)

In [15]:
t_ext = Table.read("extinction_corr_MS.csv")

GES       = ascii.read("./clusters/table3_idr6_final_eDR3.csv")
GES["BPRP"] = GES["BPmag"]-GES["RPmag"]
GES = photclean(GES, 5.)
Pleiades = ascii.read("./clusters/Pleiades_eDR3_JHKs_final.dat")
Pleiades["BPRP"] = Pleiades["BPmag"]-Pleiades["RPmag"]
Pleiades = photclean(Pleiades, 5.)
NGC2547  = ascii.read("./clusters/NGC2547_eDR3_JHKs_final.dat")
NGC2547["BPRP"] = NGC2547["BPmag"]-NGC2547["RPmag"]
NGC2547 = photclean(NGC2547, 5.)
Hyades  = ascii.read("./clusters/Hyades_Oh20_EDR3_JHKs_final.dat")
Hyades["BPRP"] = Hyades["BPmag"]-Hyades["RPmag"]
Hyades = photclean(Hyades, 5.)

/tmp/ipykernel_218411/1821515935.py:3: RuntimeWarning: invalid value encountered in power
  condphot = np.abs(phot_excess) < nsig*(0.0059898+8.817481e-12*data['Gmag']**(7.618399))


KeyError: 'BPmag'

## Choosing 2MASS or VHS
The choice of whether Ks magnitudes come from 2MASS or VHS have already been selected in the GES table. This cell simply selects whether to use 2MASS or VHS for the Jeffries+05 targets. Neither the Pleiades nor the Hyades objects are southern enough to be observed in VHS.

In [ ]:
# Minor correction to put the 2MASS data on the correct photometric scale as VHS.
# Page 11 (eqn 4.3.1) http://www.eso.org/rm/api/v1/public/releaseDescriptions/144
NGC2547["Ksmag"]  = NGC2547["Ksmag"]  + 0.01*(NGC2547["Jmag"]-NGC2547["Ksmag"])


# JUST DEAL WITH THE OBJECTS IN COMMON FOR NGC2547. Pleiades in the north so no
# VHS counterparts.
Ks_TM, Ks_VH = [], []
Ks_fin, eKs_fin, rKs_fin = [], [], []

for Vf, Tf in enumerate(NGC2547["Ksmag"]):
    Ks_T, eKs_T = NGC2547[Vf]["Ksmag"], NGC2547[Vf]["e_Ksmag"]
    Ks_V, eKs_V = NGC2547[Vf]["Ksap3"], NGC2547[Vf]["e_Ksap3"]
    if Ks_V < 12.5:
        Ks_TM.append(Ks_T)
        Ks_VH.append(Ks_V)
        Ks_fin.append(Ks_T)
        eKs_fin.append(eKs_T)
        rKs_fin.append("TB")
    if Ks_V >= 12.5:
        if eKs_V > eKs_T:
            Ks_TM.append(Ks_T)
            Ks_VH.append(Ks_V)
            Ks_fin.append(Ks_T)
            eKs_fin.append(eKs_T)
            rKs_fin.append("TF")
        else:
            Ks_TM.append(Ks_T)
            Ks_VH.append(Ks_V)
            Ks_fin.append(Ks_V)
            eKs_fin.append(eKs_V)
            rKs_fin.append("VF")

## Choosing the colours and magnitudes to plot
In an attempt to make the code generic for any set of colours/magnitudes, the following function reads in the chosen magnitude and colours (and applies a colour correction if this is Bp-Rp).

In [ ]:
def set_up_plot(tab_in, m1, m2, m3, phot_corr):
    for m in m1:
        for col in tab_in.columns:
            if m == col:
                mag_out = tab_in[m]
    for m in m2:
        for col in tab_in.columns:
            if m == col:
                m1_out = tab_in[m]
    for m in m3:
        for col in tab_in.columns:
            if m == col:
                m2_out = tab_in[m]
    if phot_corr is not None:
        for m in phot_corr:
            for col in tab_in.columns:
                if m == col:
                    cor_out = np.array(tab_in[m], dtype=float)
        col_out = m1_out - m2_out + cor_out
    else:
        col_out = m1_out - m2_out
    return mag_out, col_out

## Cleaning the parent samples.
Some basic cuts on the data to ensure high probability members are chosen.

In [ ]:
NGC2547["Ks_fin"], NGC2547["eKs_fin"] = Ks_fin, eKs_fin
NGC2547 = NGC2547[NGC2547["ruwe"]<1.4]
NGC2547.remove_columns(['Ksmag', 'e_Ksmag'])

Pleiades = Pleiades[(Pleiades["ruwe"]<1.4) & (Pleiades["pc"] >= 0.99)]
gamVel = GES[(GES["CLUSTER"]=="gamma2_Vel") & (GES["MEM3D"] >= 0.95)]
NGC6530 = GES[(GES["CLUSTER"]=="NGC6530") & (GES["MEM3D"] >= 0.95)]
NGC2516 = GES[(GES["CLUSTER"]=="NGC2516") & (GES["MEM3D"] >= 0.95)]


## Supply a choice of CMD
User needs to input whether they want the G vs G-Ks or G vs Bp-Rp CMD.

In [ ]:
x = input("Which CMD do you want? \n(1) = G vs G-Ks, (2) = G vs BP-RP")
choose_col = None
if int(x) == 1:
    choose_col = "GKs"
    m1_t = ['phot_g_mean_mag', 'Gmag', 'GMAG']
    m2_t = ['phot_g_mean_mag', 'Gmag', 'GMAG']
    m3_t = ['Kmag','Ks_fin', 'Ksmag', 'KMAGP']
    phot_corr = None
if int(x) == 2:
    choose_col = "GBPRP"    
    m1_t = ['phot_g_mean_mag','Gmag', 'GMAG']
    m2_t = ['BPmag', 'phot_bp_mean_mag']
    m3_t = ['RPmag', 'phot_rp_mean_mag']
    phot_corr = ["BPRPcorr"]

## Run the plots
Prepare the cleaning, extinction/reddening corrections, polynomial fitting and plots. These are pre-selected based on the choice of CMD. The final polynomial fits are saved as x_p2 (where x is the cluster name). The E(B-V)/d_mod are hardwired for each cluster.

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
ax.grid()
plt.gca().invert_yaxis()

NGC6530_p2,  NGC6530_col,  NGC6530_mag,  NGC6530_chi2  = prepare_cmd(choose_col,
             10.6, quad_sum(0.02, 0.09),
             0.44, 0.10, 
             set_up_plot(NGC6530, m1_t, m2_t, m3_t, phot_corr)[0],
             set_up_plot(NGC6530, m1_t, m2_t, m3_t, phot_corr)[1], 
             'gray', 'darkgray', 0.0, 'NGC 6530 ($1-2\,$Myr)', 0)
gamVel_p2,   gamVel_col,   gamVel_mag,   gamVel_chi2   = prepare_cmd(choose_col,
             7.73, quad_sum(0.01, 0.02),
             0.04, 0.03, 
             set_up_plot(gamVel, m1_t, m2_t, m3_t, phot_corr)[0], 
             set_up_plot(gamVel, m1_t, m2_t, m3_t, phot_corr)[1], 
             'limegreen', 'darkgreen', 0.0, 'gam Vel ($15-25\,$Myr)', 0)
Pleiades_p2, Pleiades_col, Pleiades_mag, Pleiades_chi2 = prepare_cmd(choose_col,
             5.65, 0.03,
             0.03, 0.01, 
             set_up_plot(Pleiades, m1_t, m2_t, m3_t, phot_corr)[0], 
             set_up_plot(Pleiades, m1_t, m2_t, m3_t, phot_corr)[1], 
             'pink', 'red', 0.0, 'Pleiades ($125\pm10\,$Myr)', 1)
NGC2547_p2,  NGC2547_col,  NGC2547_mag,  NGC2547_chi2  = prepare_cmd(choose_col,
             7.93, quad_sum(0.01, 0.03),
             0.06, 0.03, 
             set_up_plot(NGC2547, m1_t, m2_t, m3_t, phot_corr)[0], 
             set_up_plot(NGC2547, m1_t, m2_t, m3_t, phot_corr)[1],  
             'skyblue', 'blue', 0.0, 'NGC 2547 ($38-41\,$Myr)', 1)
NGC2516_p2,  NGC2516_col,  NGC2516_mag,  NGC2516_chi2  = prepare_cmd(choose_col,
             8.07, quad_sum(0.01, 0.03),
             0.11, 0.03, 
             set_up_plot(NGC2516, m1_t, m2_t, m3_t, phot_corr)[0], 
             set_up_plot(NGC2516, m1_t, m2_t, m3_t, phot_corr)[1], 
             'brown', 'brown', 0.0, 'NGC 2516 ($100-150\,$Myr)', 0)
Hyades_p2,   Hyades_col,   Hyades_mag,   Hyades_chi2   = prepare_cmd(choose_col,
             0.0, 0.010,
             0.01, 0.01, 
             set_up_plot(Hyades, m1_t, m2_t, m3_t, phot_corr)[0]-5.0*np.log10(100./Hyades["parallax"]),
             set_up_plot(Hyades, m1_t, m2_t, m3_t, phot_corr)[1], 
             'orange', 'peru', 0.0, 'Hyades ($625\pm50\,$Myr)', 1)

print(NGC2547_p2, NGC2547_chi2)
print(Pleiades_p2, Pleiades_chi2)

ax.legend()
fig.savefig(choose_col+"_CMD.jpg")

## A quick idea from Rob Jeffries
1. Fit the Pleiades polynomial to NGC2516 colours.
2. Find the fraction of NGC2516 stars that are fainter than the fit.
3. If this fraction is >5% keep shifting them bright, and find how brighter one would have to shift the Pleiades fit until it let in 95% of the NGC2516 data. It might be useful for a lower-limit to the chosen fit, however, it could also lead to contamination from much older MS stars.

In [ ]:
NGC2516_fit = make_fit(4, Pleiades_p2, NGC2516_col)

IsoDiff = NGC2516_mag - NGC2516_fit  > 0.0
n_miss = sum(bool(x) for x in IsoDiff)/len(IsoDiff)
j=0
while n_miss > 0.05:
    IsoDiff = NGC2516_mag - NGC2516_fit  > 0.0
    n_miss = sum(bool(x) for x in IsoDiff)/len(IsoDiff)
    print("n_miss = %4.2f, j = %4.2f" % (n_miss, j))
    NGC2516_mag = NGC2516_mag - 0.01
    j = j - 0.01
